# Python Code Generation

Let's take a closer look at how `mgen` generates code given a formal input specification. Starting with a data description, e.g., [common.mgen](common.mgen),
```yaml
ivals :
  nlayer : contains the @number of layers (2 or 3) in the sample
  tflash : duration of laser flash in @ms
  cps : specific heat capacities in @J kg^-1 K^-1
      - of the front face
      - of the rear face
      - of the curved side face
  rs : heat transfer coefficient for losses in  @W m^-2 K^-1
      - from front face
      - from rear face
      - curved side face
  kappas : contains radius of
      - sample (in @cm)
      - laser (in @mm)
      - measuring (in @mm)
  lams   : array of thermal conductivities of layer @nlayer (in @W m^-2 K^-1)
```
we first need to generate the appropriate Python structure for storing the composite object `ivals` and its fields. We use Python classes and the [slots](https://wiki.python.org/moin/UsingSlots) special attribute for declaring data members. We also take the opportunity to initialise array fields as empty dictionaries to avoid any `IndexError` exceptions later on.

In [1]:
class Ivals():

  __slots__ = ("lams", "kappas", "rs", "cps", "tflash", "nlayer")

  def __init__(self):
    self.lams = {}
    self.kappas = {}
    self.rs = {}
    self.cps = {}

Note that if the input specification contained nested composite objects, `mgen` would first generate the Python classes for the innermost objects. The name for these classes would be prefixed with the fully qualified scope, e.g.,
```yaml
group :
  field1 : @int
  nested_group :
    field2 : @km
```
would be translated to:

In [5]:
class GroupNested_group():

  __slots__ = ("field2")


class Group():

  __slots__ = ("nested_group", "field1")

  def __init__(self):
    self.nested_group = GroupNested_group()


Now with the needed structure in place, we are ready to start processing the data. We will be readin from the sample input text file [common.in](common.in).

In [2]:
ivals = Ivals()
fname = "common.in"

We then open the file `fname` and sequentially read the data, advancing the read pointer `readPtr` as we go along. We use the unit information to scale data into the units used internally in the program. Note also that we have merged the loops for `ivals.cps` and `ivals.rs` into a single loop, while breaking down the loop for `ivals.kappas` into a single assignment (`ivals.kappas[0] = ...`) and a loop from `1` to `2` (`for i in range(1, 3): ivals.kappas[i] = ...`) so that the different scaling factors corresponding to `cm` and `mm` are taken into account.

In [3]:
with open(fname) as f1:
  src = f1.readlines()
  readPtr = 0
  ivals.nlayer = int(src[readPtr])
  readPtr = readPtr + 1
  ivals.tflash = 1e-3 * float(src[readPtr])
  readPtr = readPtr + 1
  for i in range(0, 3):
      ivals.cps[i] = float(src[readPtr + i])
      ivals.rs[i] = 1e3 * float(src[readPtr + i + 3])
  readPtr = readPtr + 6
  ivals.kappas[0] = 1e-2 * float(src[readPtr])
  for i in range(1, 3):
      ivals.kappas[i] = 1e-3 * float(src[readPtr + i])
  readPtr = readPtr + 3
  for i in range(0, ivals.nlayer):
      ivals.lams[i] = 1e3 * float(src[readPtr + i])


Finally, we can make sure that we have read the data properly and all the attributes have been scaled correctly with respect to the unit information from the specification [common.mgen](common.mgen).

In [ ]:
def enum_inv(xs):
    return [xs[i] for i in range(0,len(xs))]
print(f"nlayer = {ivals.nlayer}")
print(f"tflash = {ivals.tflash}")
print(f"cps    = {enum_inv(ivals.cps)}")
print(f"rs     = {enum_inv(ivals.rs)}")
print(f"kappas = {enum_inv(ivals.kappas)}")
print(f"lams   = {enum_inv(ivals.lams)}")